In [ ]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importações
import os
import librosa
import soundfile as sf
import numpy as np
from pathlib import Path
import pandas as pd
import random

In [ ]:
# --- Função para adicionar ruído ---
def add_noise(y, sr, noise_factor=0.05, noise_type="white"):
    if noise_type == "white":
        noise = np.random.randn(len(y))
    elif noise_type == "pink":
        noise = np.cumsum(np.random.randn(len(y)))
        noise = noise / np.max(np.abs(noise))
    elif noise_type == "background":
        try:
            bg_noise, _ = librosa.load("background_noise.wav", sr=sr)
            noise = np.interp(np.linspace(0, len(bg_noise), len(y)), np.arange(len(bg_noise)), bg_noise)
        except Exception as e:
            print(f"Erro ao carregar ruído de fundo: {e}")
            noise = np.random.randn(len(y))
    else:
        raise ValueError("Tipo de ruído inválido. Escolha entre 'white', 'pink' ou 'background'.")

    y_noise = y + noise_factor * noise
    return np.clip(y_noise, -1.0, 1.0)

# --- Função para mudar velocidade ---
def change_speed(y, sr, rate=1.0):
    try:
        if len(y) < 2:
            raise ValueError("Áudio muito curto para alteração de velocidade.")
        y = y.astype(np.float32)
        D = librosa.stft(y)
        D_stretched = librosa.phase_vocoder(D, rate=rate)
        y_stretched = librosa.istft(D_stretched)
        return y_stretched
    except Exception as e:
        print(f"Erro ao alterar velocidade: {e}")
        return y

# --- Função para mudar pitch manualmente ---
def change_pitch_manual(y, sr, n_steps=2):
    try:
        return librosa.effects.pitch_shift(y, sr, n_steps)
    except Exception as e:
        print(f"Erro ao alterar pitch: {e}")
        return y

# --- Função para aplicar equalização aleatória ---
def random_equalization(y, sr, n_bands=5, max_gain_db=12):
    frequencies = np.linspace(0, sr // 2, n_bands + 2)
    gains_db = np.random.uniform(-max_gain_db, max_gain_db, n_bands + 2)
    gains_linear = 10 ** (gains_db / 20)

    y_eq = np.zeros_like(y)
    for i in range(len(frequencies) - 1):
        y_eq += y * gains_linear[i] / (len(frequencies) - 1)
    return np.clip(y_eq, -1.0, 1.0)

# --- Função SpecAugment para MFCCs ---
def spec_augment(mfccs, time_mask_param=10, freq_mask_param=5):
    mfccs_aug = mfccs.copy()
    t = mfccs.shape[1]
    t0 = np.random.randint(0, max(1, t - time_mask_param))
    mfccs_aug[:, t0:t0 + time_mask_param] = 0

    f = mfccs.shape[0]
    f0 = np.random.randint(0, max(1, f - freq_mask_param))
    mfccs_aug[f0:f0 + freq_mask_param, :] = 0

    return mfccs_aug

# --- Função para extrair MFCCs ---
def extract_features(y, sr):
    try:
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Erro ao extrair MFCCs: {e}")
        return None

# --- Aplica UMA técnica por arquivo (para dobrar no máximo a base) ---
def load_data_augmented_multiple(folder_path, label, output_path, n_aug=3,
                                 noise_type="white", speed_rate=0.85, pitch_steps=2,
                                 time_mask_param=10, freq_mask_param=5):
    data = []
    labels = []
    os.makedirs(output_path, exist_ok=True)
    file_list = sorted(os.listdir(folder_path))

    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        try:
            y, sr = librosa.load(file_path, sr=None)
            base_name = os.path.splitext(file_name)[0]

            # Original
            features_orig = extract_features(y, sr)
            if features_orig is not None:
                data.append(features_orig)
                labels.append(label)
                sf.write(os.path.join(output_path, f"{base_name}_original.wav"), y, sr)

            for i in range(n_aug):
                technique = random.choice(["noise", "speed", "pitch", "equalization", "spec"])

                if technique == "noise":
                    y_aug = add_noise(y, sr, noise_type=noise_type)
                    features_aug = extract_features(y_aug, sr)
                    aug_name = f"{base_name}_noise_{noise_type}_{i}.wav"

                elif technique == "speed":
                    y_aug = change_speed(y, sr, rate=speed_rate)
                    features_aug = extract_features(y_aug, sr)
                    aug_name = f"{base_name}_speed_{speed_rate}_{i}.wav"

                elif technique == "pitch":
                    y_aug = change_pitch_manual(y, sr, n_steps=pitch_steps)
                    features_aug = extract_features(y_aug, sr)
                    aug_name = f"{base_name}_pitch_{pitch_steps}_{i}.wav"

                elif technique == "equalization":
                    y_aug = random_equalization(y, sr)
                    features_aug = extract_features(y_aug, sr)
                    aug_name = f"{base_name}_eq_{i}.wav"

                elif technique == "spec":
                    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
                    mfccs_aug = spec_augment(mfccs, time_mask_param, freq_mask_param)
                    features_aug = np.mean(mfccs_aug.T, axis=0)
                    y_aug = None
                    aug_name = None

                if features_aug is not None:
                    data.append(features_aug)
                    labels.append(label)
                    if y_aug is not None and aug_name:
                        sf.write(os.path.join(output_path, aug_name), y_aug, sr)

        except Exception as e:
            print(f"Erro ao processar {file_path}: {e}")

    return data, labels

In [ ]:
file_path_real = '/content/drive/MyDrive/audios_FOR/real'
file_path_fake = '/content/drive/MyDrive/audios_FOR/fake'

out_augmented_real = '/content/drive/MyDrive/audios_FOR/real_augmented_all'
out_augmented_fake = '/content/drive/MyDrive/audios_FOR/fake_augmented_all'

data_real, labels_real = load_data_augmented_multiple(
    folder_path=file_path_real,
    label=1,
    n_aug=3,  # ← Aumente este número para gerar mais dados
    noise_type="pink",
    speed_rate=0.85,
    pitch_steps=2,
    time_mask_param=10,
    freq_mask_param=5,
    output_path=out_augmented_real
)

data_fake, labels_fake = load_data_augmented_multiple(
    folder_path=file_path_fake,
    label=0,
    n_aug=3,
    noise_type="white",
    speed_rate=0.85,
    pitch_steps=2,
    time_mask_param=10,
    freq_mask_param=5,
    output_path=out_augmented_fake
)

# Concatenar e filtrar features válidas
data = data_real + data_fake
labels = labels_real + labels_fake
data_filtered, labels_filtered = zip(*[(d, l) for d, l in zip(data, labels) if d is not None])

Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar pitch: pitch_shift() takes 1 positional argument but 3 were given
Erro ao alterar 

In [ ]:
# Criar DataFrame
df = pd.DataFrame(data_filtered)
df['label'] = labels_filtered

In [ ]:
df.shape

(2880, 14)